Loaded data and extracted pairs

In [3]:
from xml.etree import ElementTree as ET
from lxml import etree
import os

In [4]:
fileNames=[]

In [5]:
for file in os.listdir('../../Nachweise/'):
    tree = ET.parse('../../Nachweise/'+file)
    root=tree.getroot()
    children=[]
    for child in root:
        children.append(child.tag)
    if len(children)==4:
        fileNames.append(file)

In [6]:
len(fileNames)

42

In [7]:
fileNames=sorted(fileNames)

In [8]:
dictionaryOfReferences=dict()

In [9]:
for file in fileNames:
    text = open('../../Nachweise/'+file).read()
    root = etree.fromstring(text)[1]
    ni=''.join(root.itertext())
    ni=ni.replace('\t','')
    ni=ni.replace('\n','')
    
    root = etree.fromstring(text)[2]
    cite=''.join(root.itertext())
    cite=cite.replace('\t','')
    cite=cite.replace('\n','')
    
    root = etree.fromstring(text)[3]
    ref=[root.text]
    for i in root.iterdescendants():
        if i.tag=='fn':
            break
        if i.text!=None:
            ref.append(i.text)
        if i.tail!=None:
            ref.append(i.tail)
    ref=''.join(ref)
    ref=ref.replace('\t','')
    ref=ref.replace('\n','')
    
    dictionaryOfReferences[file]=[ni,cite,ref]
    
    
    

Extracting metrics from pairs

In [12]:
import numpy as np
from nltk.corpus import wordnet as wn
from stanfordcorenlp import StanfordCoreNLP
import re
import bisect
from collections import defaultdict
import ast
import os
from gutenberg.cleanup import strip_headers
from nltk.tokenize import sent_tokenize
from bs4 import BeautifulSoup
import math
import gensim
import pickle
from scipy import spatial
from nltk.tree import *
import nltk.corpus
import nltk.tokenize.punkt
import nltk.stem.snowball
import string
from multiprocessing import Pool
from nltk.draw.tree import TreeView
from fuzzywuzzy import fuzz
from multiprocessing import Pool
from nltk import word_tokenize,pos_tag
from nltk.corpus import wordnet 
from operator import itemgetter

In [29]:
public='/home/users2/mehrotsh/scripts/packages/stanford-corenlp-full-2018-02-27/'
personal='/home/samarth/stanford-corenlp-full-2018-02-27/'

In [30]:
nlp = StanfordCoreNLP(public)

Exception ignored in: <socket.socket fd=42, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 37864), raddr=('127.0.0.1', 9000)>


In [31]:
def tree(): 
    return defaultdict(tree)


def _leadingSpaces_(target):
    return len(target) - len(target.lstrip())

def _findParent_(curIndent, parid, treeRef):
    tmpid = parid
    while (curIndent <= treeRef[tmpid]['indent']):
        tmpid = treeRef[tmpid]['parid']
    return tmpid


def generateTree(rawTokens, treeRef):

    # (token
    REGEX_OPEN = r"^\s*\(([a-zA-Z0-9_']*)\s*$"
    # (token (tok1 tok2) (tok3 tok4) .... (tokx toky))
    REGEX_COMP = r"^\s*\(([a-zA-Z0-9_']+)\s*((?:[(]([a-zA-Z0-9_;.,?'!]+)\s*([a-zA-Z0-9_;\.,?!']+)[)]\s*)+)"    
    # (, ,) as stand-alone. Used for match() not search()
    REGEX_PUNC = r"^\s*\([,!?.'\"]\s*[,!?.'\"]\)"
    # (tok1 tok2) as stand-alone
    REGEX_SOLO_PAIR = r"^\s*\(([a-zA-Z0-9_']+)\s*([a-zA-Z0-9_']+)\)"
    # (tok1 tok2) used in search()
    REGEX_ISOL_IN_COMP = r"\(([a-zA-Z0-9_;.,?!']+)\s*([a-zA-Z0-9_;.,?!']+)\)"
    # (punc punc) used in search()
    REGEX_PUNC_SOLO = r"\([,!?.'\"]\s*[,!?.'\"]\)"
   
    treeRef[len(treeRef)] = {'curid':0, 
                             'parid':-1, 
                             'posOrTok':'ROOT', 
                             'indent':0,
                            'children':[],
                            'childrenTok':[]}
    ID_CTR = 1
    
    for tok in rawTokens[1:]:
        
        curIndent = _leadingSpaces_(tok) 
        parid = _findParent_(curIndent, ID_CTR-1, treeRef)
        
        # CHECK FOR COMPOSITE TOKENS
        checkChild = re.match(REGEX_COMP, tok)
        if (checkChild):
            treeRef[ID_CTR] = {'curid':ID_CTR, 
                               'parid':parid, 
                               'posOrTok':checkChild.group(1), 
                               'indent':curIndent,
                              'children':[],
                              'childrenTok':[]}
            upCTR = ID_CTR
            ID_CTR += 1
            
            subCheck = re.sub(REGEX_PUNC_SOLO,'',checkChild.group(2))
            subs = re.findall(REGEX_ISOL_IN_COMP, subCheck) 
            for ch in subs:
                treeRef[ID_CTR] = {'curid':ID_CTR, 
                                   'parid':upCTR, 
                                   'posOrTok':ch[0], 
                                   'indent':curIndent+2,
                                  'children':[],
                                  'childrenTok':[]}
                ID_CTR += 1
                treeRef[ID_CTR] = {'curid':ID_CTR, 
                                   'parid':ID_CTR-1, 
                                   'posOrTok':ch[1], 
                                   'indent':curIndent+2,
                                  'children':[],
                                  'childrenTok':[]}
                ID_CTR += 1
            continue
           

            
        checkSingle = re.match(REGEX_SOLO_PAIR, tok)
        if (checkSingle):
            treeRef[ID_CTR] = {'curid':ID_CTR, 
                               'parid':parid, 
                               'posOrTok':checkSingle.group(1), 
                               'indent':curIndent+2,
                              'children':[],
                              'childrenTok':[]}
            ID_CTR += 1
            treeRef[ID_CTR] = {'curid':ID_CTR, 
                               'parid':ID_CTR-1, 
                               'posOrTok':checkSingle.group(2), 
                               'indent':curIndent+2,
                              'children':[],
                              'childrenTok':[]}
            ID_CTR += 1
            continue
        
        
        checkPunc = re.match(REGEX_PUNC, tok)
        if (checkPunc): # ignore punctuation
            continue

        checkMatch = re.match(REGEX_OPEN, tok)
        if (checkMatch):
            treeRef[ID_CTR] = {'curid':ID_CTR, 
                               'parid':parid, 
                               'posOrTok':checkMatch.group(1), 
                               'indent':curIndent,
                              'children':[],
                              'childrenTok':[]}
            ID_CTR += 1
            continue

    return
            

def flipTree(treeRef):
    # Pass 1 fill in children
    for k,v in treeRef.items():
        if (k > 0):
            bisect.insort(treeRef[v['parid']]['children'], k)
    # Pass 2 map children to tokens
    for k,v in treeRef.items():
        if (k > 0):
            treeRef[k]['childrenTok'] = [treeRef[ch]['posOrTok'] for ch in treeRef[k]['children']]
    treeRef[0]['childrenTok'] = treeRef[1]['posOrTok']


In [32]:
def _isLeaf_(tree, parentNode):
    return (len(tree[parentNode]['children']) == 0)

def _isPreterminal_(tree, parentNode):
    for idx in tree[parentNode]['children']:
        if not _isLeaf_(tree, idx):
            return False
    return True

'''
Implementation of the Colins-Duffy or Subset-Tree (SST) Kernel
'''

def _cdHelper_(tree1, tree2, node1, node2, store, lam, SST_ON):
    # No duplicate computations
    if store[node1, node2] >= 0:
        return

    # Leaves yield similarity score by definition
    if (_isLeaf_(tree1, node1) or _isLeaf_(tree2, node2)):
        store[node1, node2] = 0
        return

    # same parent node
    if tree1[node1]['posOrTok'] == tree2[node2]['posOrTok']:
        # same children tokens
        if tree1[node1]['childrenTok'] == tree2[node2]['childrenTok']:
            # Check if both nodes are pre-terminal
            if _isPreterminal_(tree1, node1) and _isPreterminal_(tree2, node2):
                store[node1, node2] = lam
                return
            # Not pre-terminal. Recurse among the children of both token trees.
            else:
                nChildren = len(tree1[node1]['children'])

                runningTotal = None
                for idx in range(nChildren):
                     # index ->  node_id
                    tmp_n1 = tree1[node1]['children'][idx]
                    tmp_n2 = tree2[node2]['children'][idx]
                    # Recursively run helper
                    _cdHelper_(tree1, tree2, tmp_n1, tmp_n2, store, lam, SST_ON)
                    # Set the initial value for the layer. Else multiplicative product.
                    if (runningTotal == None):
                        runningTotal = SST_ON + store[tmp_n1, tmp_n2]
                    else:
                        runningTotal *= (SST_ON + store[tmp_n1, tmp_n2])

                store[node1, node2] = lam * runningTotal
                return
        else:
            store[node1, node2] = 0
    else: # parent nodes are different
        store[node1, node2] = 0
        return


def _cdKernel_(tree1, tree2, lam, SST_ON):
    # Fill the initial state of the store
    store = np.empty((len(tree1), len(tree2)))
    store.fill(-1)
    # O(N^2) to compute the tree dot product
    for i in range(len(tree1)):
        for j in range(len(tree2)):
            _cdHelper_(tree1, tree2, i, j, store, lam, SST_ON)

    return store.sum()

'''
Returns a tuple w/ format: (raw, normalized)
If NORMALIZE_FLAG set to False, tuple[1] = -1
'''
def CollinsDuffy(tree1, tree2, lam, NORMALIZE_FLAG, SST_ON):
    raw_score = _cdKernel_(tree1, tree2, lam, SST_ON)
    if (NORMALIZE_FLAG):
        t1_score = _cdKernel_(tree1, tree1, lam, SST_ON)
        t2_score = _cdKernel_(tree2, tree2, lam, SST_ON)
        return (raw_score,(raw_score / math.sqrt(t1_score * t2_score)))
    else:
        return (raw_score,-1)



'''
Implementation of the Partial Tree (PT) Kernel from:
"Efficient Convolution Kernels for Dependency and Constituent Syntactic Trees"
by Alessandro Moschitti
'''

'''
The delta function is stolen from the Collins-Duffy kernel
'''

def _deltaP_(tree1, tree2, seq1, seq2, store, lam, mu, p):

#     # Enumerate subsequences of length p+1 for each child set
    if p == 0:
        return 0
    else:
        # generate delta(a,b)
        _delta_(tree1, tree2, seq1[-1], seq2[-1], store, lam, mu)
        if store[seq1[-1], seq2[-1]] == 0:
            return 0
        else:
            runningTot = 0
            for i in range(p-1, len(seq1)-1):
                for r in range(p-1, len(seq2)-1):
                    scaleFactor = pow(lam, len(seq1[:-1])-i+len(seq2[:-1])-r)
                    dp = _deltaP_(tree1, tree2, seq1[:i], seq2[:r], store, lam, mu, p-1)
                    runningTot += (scaleFactor * dp)
            return runningTot

def _delta_(tree1, tree2, node1, node2, store, lam, mu):

    # No duplicate computations
    if store[node1, node2] >= 0:
        return

    # Leaves yield similarity score by definition
    if (_isLeaf_(tree1, node1) or _isLeaf_(tree2, node2)):
        store[node1, node2] = 0
        return

    # same parent node
    if tree1[node1]['posOrTok'] == tree2[node2]['posOrTok']:

        if _isPreterminal_(tree1, node1) and _isPreterminal_(tree2, node2):
            if tree1[node1]['childrenTok'] == tree2[node2]['childrenTok']:
                store[node1, node2] = lam
            else:
                store[node1, node2] = 0
            return

        else:
            # establishes p_max
            childmin = min(len(tree1[node1]['children']), len(tree2[node2]['children']))
            deltaTot = 0
            for p in range(1,childmin+1):
                # compute delta_p
                deltaTot += _deltaP_(tree1, tree2,
                                     tree1[node1]['children'],
                                     tree2[node2]['children'], store, lam, mu, p)

            store[node1, node2] = mu * (pow(lam,2) + deltaTot)
            return

    else:
        # parent nodes are different
        store[node1, node2] = 0
        return

def _ptKernel_(tree1, tree2, lam, mu):
    # Fill the initial state of the store
    store = np.empty((len(tree1), len(tree2)))
    store.fill(-1)

    # O(N^2) to compute the tree dot product
    for i in range(len(tree1)):
        for j in range(len(tree2)):
            _delta_(tree1, tree2, i, j, store, lam, mu)

    return store.sum()

'''
Returns a tuple w/ format: (raw, normalized)
If NORMALIZE_FLAG set to False, tuple[1] = -1
'''
def MoschittiPT(tree1, tree2, lam, mu, NORMALIZE_FLAG):
    raw_score = _ptKernel_(tree1, tree2, lam, mu)
    if (NORMALIZE_FLAG):
        t1_score = _ptKernel_(tree1, tree1, lam, mu)
        t2_score = _ptKernel_(tree2, tree2, lam, mu)
        return (raw_score,(raw_score / math.sqrt(t1_score * t2_score)))
    else:
        return (raw_score,-1)

In [33]:
def getNLPToks(rawSentence):
    output = nlp.annotate(rawSentence, properties={'annotators': 'tokenize,ssplit,pos,parse','outputFormat': 'json','timeout':'50000','pipelineLanguage': 'german'})
    output=ast.literal_eval(output)
    tokens = output['sentences'][0]['tokens']
    parse = output['sentences'][0]['parse'].split("\n")
    return {
        'toks':tokens, 'parse':parse
    }

In [34]:
def getMoschittiScore(sent1,sent2):
    tree_1=tree()
    tree_2=tree()
    out1=getNLPToks(sent1)
    out2=getNLPToks(sent2)
    generateTree(out1['parse'],tree_1)
    generateTree(out2['parse'],tree_2)
    flipTree(tree_1)
    flipTree(tree_2)
    (rscore_st, nscore_st) = MoschittiPT(tree_1, tree_2, 0.8, 1, 1)
#     return rscore_st,nscore_st
    return nscore_st

In [35]:
def avg_feature_vector(sentence, model, num_features, index2word_set):
    words=word_tokenize(sentence)
    # words = sentence.split()
    # words = [token.lower().strip(string.punctuation) for token in tokenizer.tokenize(sentence) if token.lower().strip(string.punctuation) not in stopwords]
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec  

def avg_feature_vector_without_stopwords(sentence, model, num_features, index2word_set):
    words=word_tokenize(sentence)
    # words = sentence.split()
    words = [token.lower().strip(string.punctuation) for token in words if token.lower().strip(string.punctuation) not in stopwords]
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec  

def avg_feature_vector_nouns(sentence, model, num_features, index2word_set):
    words=word_tokenize(sentence)
    # words = sentence.split()
    # words = [token.lower().strip(string.punctuation) for token in tokenizer.tokenize(sentence) if token.lower().strip(string.punctuation) not in stopwords]
    nouns=[]
    for word,pos in nltk.pos_tag(words):
        if pos.startswith('NN'):
            nouns.append(word.lower().strip(string.punctuation))   

    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in nouns:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec  

def avg_feature_vector_verbs(sentence, model, num_features, index2word_set):
    words=word_tokenize(sentence)
    # words = sentence.split()
    # words = [token.lower().strip(string.punctuation) for token in tokenizer.tokenize(sentence) if token.lower().strip(string.punctuation) not in stopwords]
    verbs=[]
    for word,pos in nltk.pos_tag(words):
        if pos.startswith('VB'):
            verbs.append(word.lower().strip(string.punctuation))   

    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in verbs:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec  


def jacardNouns(sent1,sent2):
    nouns1=[]
    for word,pos in nltk.pos_tag(word_tokenize(sent1)):
        if pos.startswith('NN'):
            nouns1.append(word.lower().strip(string.punctuation))
    nouns2=[]
    for word,pos in nltk.pos_tag(word_tokenize(sent2)):
        if pos.startswith('NN'):
            nouns2.append(word.lower().strip(string.punctuation))
#     print(nouns1)
#     print(nouns2)
    if len(set(nouns1).union(nouns2))==0:
        ratio=0
    else:
        ratio = len(set(nouns1).intersection(nouns2)) / float(len(set(nouns1).union(nouns2)))        
    return ratio

def jacardVerbs(sent1,sent2):
    nouns1=[]
    for word,pos in nltk.pos_tag(word_tokenize(sent1)):
        if pos.startswith('VB'):
            nouns1.append(word.lower().strip(string.punctuation))
    nouns2=[]
    for word,pos in nltk.pos_tag(word_tokenize(sent2)):
        if pos.startswith('VB'):
            nouns2.append(word.lower().strip(string.punctuation))
#     print(nouns1)
#     print(nouns2)
    if len(set(nouns1).union(nouns2))==0:
        ratio=0
    else:
        ratio = len(set(nouns1).intersection(nouns2)) / float(len(set(nouns1).union(nouns2)))        
    return ratio

def jacardAdj(sent1,sent2):
    nouns1=[]
    for word,pos in nltk.pos_tag(word_tokenize(sent1)):
        if pos.startswith('JJ'):
            nouns1.append(word.lower().strip(string.punctuation))
    nouns2=[]
    for word,pos in nltk.pos_tag(word_tokenize(sent2)):
        if pos.startswith('JJ'):
            nouns2.append(word.lower().strip(string.punctuation))
#     print(nouns1)
#     print(nouns2)
    if len(set(nouns1).union(nouns2))==0:
        ratio=0
    else:
        ratio = len(set(nouns1).intersection(nouns2)) / float(len(set(nouns1).union(nouns2)))        
    return ratio



def longestSubsequence(a, b):
    a=a.split()
    b=b.split()
    lengths = [[0 for j in range(len(b)+1)] for i in range(len(a)+1)]
    # row 0 and column 0 are initialized to 0 already
    for i, x in enumerate(a):
        for j, y in enumerate(b):
            if x == y:
                lengths[i+1][j+1] = lengths[i][j] + 1
            else:
                lengths[i+1][j+1] = max(lengths[i+1][j], lengths[i][j+1])
    # read the substring out from the matrix
    result = ""
    x, y = len(a), len(b)
    while x != 0 and y != 0:
        if lengths[x][y] == lengths[x-1][y]:
            x -= 1
        elif lengths[x][y] == lengths[x][y-1]:
            y -= 1
        else:
            assert a[x-1] == b[y-1]
            result = a[x-1] + " " +result
            x -= 1
            y -= 1
    return result



def jacardScore(a, b):
    tokens_a = [token.lower().strip(string.punctuation) for token in tokenizer.tokenize(a) if token.lower().strip(string.punctuation) not in stopwords]
    tokens_b = [token.lower().strip(string.punctuation) for token in tokenizer.tokenize(b) if token.lower().strip(string.punctuation) not in stopwords]
    if len(set(tokens_a).union(tokens_b))==0:
        ratio=0
    else:
        ratio = len(set(tokens_a).intersection(tokens_b)) / float(len(set(tokens_a).union(tokens_b)))
    return ratio    


In [36]:
stopwords = nltk.corpus.stopwords.words('german')
stopwords.extend(string.punctuation)

Exception ignored in: <_io.FileIO name='/home/users2/mehrotsh/nltk_data/corpora/stopwords/german' mode='rb' closefd=True>


In [37]:
import warnings
warnings.filterwarnings("error")

In [23]:
model = gensim.models.KeyedVectors.load_word2vec_format('/home/users2/mehrotsh/Downloads/german.model', binary=True) 
index2word_set = set(model.wv.index2word)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [38]:
for key in dictionaryOfReferences.keys():
    ref=dictionaryOfReferences[key]
    ni=ref[0]
    r=ref[2]
    ni_sents=sent_tokenize(ni)
    r_sents=sent_tokenize(r)
    s=0
    i=0
    for sent_ni in ni_sents:
        for sent_r in r_sents:
            try:
                nscore_st = getMoschittiScore(sent_ni,sent_r)
            except RuntimeWarning:
                nscore_st=0
            s=s+nscore_st
            i=i+1
    avgSyntacticScore=s/i
    
    niv=avg_feature_vector(ni,model,300,index2word_set)
    ni_withoutStopWords=avg_feature_vector_without_stopwords(ni,model,300,index2word_set)
    ni_nouns=avg_feature_vector_nouns(ni,model,300,index2word_set)
    ni_verbs=avg_feature_vector_verbs(ni,model,300,index2word_set)
    
    
    rv=avg_feature_vector(r,model,300,index2word_set)
    r_withoutStopWords=avg_feature_vector_without_stopwords(r,model,300,index2word_set)
    r_nouns=avg_feature_vector_nouns(r,model,300,index2word_set)
    r_verbs=avg_feature_vector_verbs(r,model,300,index2word_set)

    try:
        semScore=1 - spatial.distance.cosine(niv, rv)
    except RuntimeWarning:
        semScore=0
    try:
        semScore_withoutStop=1 - spatial.distance.cosine(ni_withoutStopWords, r_verbs)
    except RuntimeWarning:
        semScore_withoutStop=0
    try:
        semScore_nouns=1 - spatial.distance.cosine(ni_nouns, r_verbs)
    except RuntimeWarning:
        semScore_nouns=0
    try:
        semScore_verbs=1 - spatial.distance.cosine(ni_verbs, r_verbs)   
    except RuntimeWarning:
        semScore_verbs=0
         
    jaccard=jacardScore(ni,r)
    jaccard_nouns=jacardNouns(ni,r)
    jaccard_verbs=jacardVerbs(ni,r)
    jaccard_adj=jacardAdj(ni,r)
    
    tup=(avgSyntacticScore,semScore,semScore,semScore_nouns,semScore_verbs,
        jaccard,jaccard_nouns,jaccard_verbs,jaccard_adj)

    print(tup)

(0.4418897675483576, 0.355144202709198, 0.355144202709198, 0.4445374608039856, 0, 0.013333333333333334, 0.0, 0.0, 0.0)
(0.7321045487182407, 0.427890419960022, 0.427890419960022, 0.21523945033550262, 0, 0.02631578947368421, 0.0, 0.0, 0.0)
(0.6922643636088055, 0.8379891514778137, 0.8379891514778137, 0.718041181564331, 0, 0.03496503496503497, 0.06962025316455696, 0.0, 0.05)
(0.6438726881801184, 0.8994180560112, 0.8994180560112, 0.5706943273544312, 0.270663857460022, 0.1320754716981132, 0.1323529411764706, 0.125, 0.16666666666666666)
(0.6990536284778589, 0.42262688279151917, 0.42262688279151917, 0.23155692219734192, 0, 0.014705882352941176, 0.018518518518518517, 0.0, 0.0)
(0.7760662827350492, 0.36826932430267334, 0.36826932430267334, 0.27454882860183716, 0.1815979778766632, 0.012658227848101266, 0.0, 0.0, 0.0)
(0.831709341200051, 0.8423121571540833, 0.8423121571540833, 0.5226263403892517, 0.5096070766448975, 0.013888888888888888, 0.045454545454545456, 0.0, 0.125)
(0.5676502369757287, 0.651

KeyboardInterrupt: 